# Kerasの使い方

## 参考資料

 - [詳解ディープラーニング 第2版 (Compass Books)](https://www.amazon.co.jp/gp/product/B081RN1Z9C/ref=ppx_yo_dt_b_d_asin_title_o00?ie=UTF8&psc=1)
 - [TensorFlow, Kerasの基本的な使い方（モデル構築・訓練・評価・予測） | note.nkmk.me](https://note.nkmk.me/python-tensorflow-keras-basics/)
 - [Keras: Pythonの深層学習ライブラリ](https://keras.io/ja/)

## データの準備

In [73]:
import pandas as pd
from sklearn.datasets import fetch_california_housing, load_breast_cancer


住宅価格の特徴量, カルフォルニアの住宅価格 = fetch_california_housing(return_X_y=True)
カルフォルニアの住宅価格_ser = pd.Series(カルフォルニアの住宅価格, name="カルフォルニアの住宅価格")
住宅価格の特徴量_df = pd.DataFrame(住宅価格の特徴量)

In [74]:
from sklearn.model_selection import train_test_split


train_feature_df, test_feature_df, train_teacher_ser, test_teacher_ser = train_test_split(住宅価格の特徴量_df, カルフォルニアの住宅価格_ser)

## モデルの構築 

 - [活性化関数 - Keras Documentation](https://keras.io/ja/activations/)
 - [正則化 - Keras Documentation](https://keras.io/ja/regularizers/)
 - [レイヤーの重み初期化 - Keras Documentation](https://keras.io/ja/initializers/)

### Sequential API : シンプルな一直線のモデルを構築可能

In [76]:
import tensorflow as tf

回帰特徴量の数 = train_住宅価格の特徴量_df.shape[1]
回帰モデルの名前 = 'model_name'

回帰モデル = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(回帰特徴量の数, ), name="input_layer"),
    tf.keras.layers.Dense(units=10, activation='linear'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=10, activation='linear'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='relu')
], name=回帰モデルの名前)


分類特徴量の数 = 10
分類モデルの名前 = 'classifier'

分類モデル = tf.keras.Sequential(name=分類モデルの名前)
分類モデル.add(tf.keras.layers.Flatten(input_shape=(回帰特徴量の数, )))  # 入力層
分類モデル.add(tf.keras.layers.Dense(3, activation='sigmoid'))  # 隠れ層
分類モデル.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 出力層

In [77]:
回帰モデル.summary()

Model: "model_name"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Flatten)        (None, 8)                 0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                90        
_________________________________________________________________
dropout_20 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_21 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 211
Trainable params: 211
Non-trainable params: 0
__________________________________________________________

In [78]:
分類モデル.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 4         
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


### Subclassing API (Model Subclassing) : 最も柔軟にモデルを構築可能

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(10, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        return x

回帰モデル = MyModel()

## 学習プロセスの設定

 - [最適化 - Keras Documentation](https://keras.io/ja/optimizers/)
 - [損失関数 - Keras Documentation](https://keras.io/ja/losses/)
 - [評価関数 - Keras Documentation](https://keras.io/ja/metrics/)

In [63]:
最適化アルゴリズム = 'adam'
損失関数 = 'mean_squared_error'
評価関数_list  = ['mse', 'mae']

回帰モデル.compile(
    optimizer=最適化アルゴリズム,
    loss=損失関数,
    metrics=評価関数_list
)

## 学習

 - [コールバック - Keras Documentation](https://keras.io/ja/callbacks/)

In [64]:
from keras.callbacks import EarlyStopping


バッチサイズ = 200
エポック数 = 20
コールバックs = [
    EarlyStopping(monitor='val_loss')
]

history = 回帰モデル.fit(
    train_feature_df, train_teacher_ser, 
    batch_size=バッチサイズ,
    epochs=エポック数,
    callbacks=コールバックs
)
history

Epoch 1/20
78/78 [==============================] - 0s 3ms/step - loss: 26994.6094 - mse: 26994.6094 - mae: 38.7897
Epoch 2/20
78/78 [==============================] - 0s 4ms/step - loss: 3145.8533 - mse: 3145.8533 - mae: 7.7134
Epoch 3/20
78/78 [==============================] - 0s 2ms/step - loss: 866.3813 - mse: 866.3813 - mae: 4.9079
Epoch 4/20
78/78 [==============================] - 0s 3ms/step - loss: 983.8842 - mse: 983.8842 - mae: 4.5702
Epoch 5/20
78/78 [==============================] - 0s 2ms/step - loss: 328.3361 - mse: 328.3361 - mae: 3.5879
Epoch 6/20
78/78 [==============================] - 0s 3ms/step - loss: 344.6814 - mse: 344.6814 - mae: 3.2991
Epoch 7/20
78/78 [==============================] - 0s 3ms/step - loss: 316.5916 - mse: 316.5916 - mae: 3.2988
Epoch 8/20
78/78 [==============================] - 0s 3ms/step - loss: 92.9530 - mse: 92.9530 - mae: 2.6984
Epoch 9/20
78/78 [==============================] - 0s 2ms/step - loss: 176.1395 - mse: 176.1395 - mae: 2.8

In [65]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 9))
plt.title("Graph of Loss function value")
plt.plot(range(エポック数), history.history['loss'], marker='.', label='loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

<Figure size 1800x900 with 1 Axes>

## 評価

In [66]:
metrics = 回帰モデル.evaluate(test_feature_df, test_teacher_ser)

162/162 [==============================] - 0s 2ms/step - loss: 7.8646 - mse: 7.8646 - mae: 2.1216


In [67]:
pd.Series(
    metrics,
    index=回帰モデル.metrics_names,
    name='評価指標'
)

loss    7.864568
mse     7.864568
mae     2.121597
Name: 評価指標, dtype: float64

## 予測

In [68]:
predicted_y_arr = 回帰モデル.predict(test_feature_df)

In [69]:
predicted_y_df = pd.DataFrame(predicted_y_arr)
predicted_y_df.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [132]:
predicted_y_df[0].value_counts()

0.0    5160
Name: 0, dtype: int64

## 保存

In [70]:
回帰モデル.save('/Users/taiyou/Desktop/kerasの使い方モデル.h5')

## 復元

In [71]:
import tensorflow as tf


model = tf.keras.models.load_model('/Users/taiyou/Desktop/kerasの使い方モデル.h5')

In [72]:
model.predict(test_feature_df)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)